In [3]:
#import libraries
from os import makedirs 
from os import listdir 
from shutil import copyfile 
from random import seed 
from random import random
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Input
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.models import Model
from keras.layers import Dropout
from keras.applications.vgg16 import VGG16
from keras.optimizers import Adam

Using TensorFlow backend.


In [4]:
# taking VGG16 model,removing the classifier part and training classifier according to our data
def model():
    model = VGG16(include_top = False,input_shape = (224,224,3))
    for layers in model.layers:
        layers.trainable = False
    flat = Flatten()(model.layers[-1].output)
    dense1 = Dense(512,activation='relu')(flat)
    drop1 = Dropout(0.5)(dense1)
    dense2 = Dense(512,activation='relu')(drop1)
    drop2 = Dropout(0.5)(dense2)
    output = Dense(1,activation='sigmoid')(drop2)
    model = Model(model.inputs,output)
    return model

In [5]:
model = model()
model.compile(optimizer = Adam(lr = 0.0001),loss = 'binary_crossentropy',metrics = ['accuracy'])

In [6]:
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0   

In [7]:
train_datagen = ImageDataGenerator(
    
    rescale=1./255,
    horizontal_flip=True,
    width_shift_range=0.1,
    height_shift_range=0.1,
    featurewise_center = True
)

train_datagen.mean = [123.68, 116.779, 103.939]

validation_datagen = ImageDataGenerator(rescale=1./255,featurewise_center=True)
validation_datagen.mean = [123.68, 116.779, 103.939]

In [8]:
train = train_datagen.flow_from_directory('C:/Users/Arun/dataset_dogs_vs_cats/train',target_size=(224,224),class_mode='binary',batch_size=  64)
validation = validation_datagen.flow_from_directory('C:/Users/Arun/dataset_dogs_vs_cats/test',target_size=(224,224),class_mode='binary',batch_size = 64)

Found 18697 images belonging to 2 classes.
Found 6303 images belonging to 2 classes.


In [9]:
history = model.fit_generator(train,steps_per_epoch=len(train),epochs=50)

Epoch 1/50
293/293 [==============================] - 250s 852ms/step - loss: 0.7393 - accuracy: 0.5042
Epoch 2/50
293/293 [==============================] - 251s 855ms/step - loss: 0.6932 - accuracy: 0.5016
Epoch 3/50
293/293 [==============================] - 246s 841ms/step - loss: 0.6933 - accuracy: 0.5006
Epoch 4/50
293/293 [==============================] - 237s 808ms/step - loss: 0.6932 - accuracy: 0.5014
Epoch 5/50
293/293 [==============================] - 239s 816ms/step - loss: 0.6934 - accuracy: 0.4995
Epoch 6/50
293/293 [==============================] - 250s 853ms/step - loss: 0.6933 - accuracy: 0.4966
Epoch 7/50
293/293 [==============================] - 250s 853ms/step - loss: 0.6932 - accuracy: 0.5017
Epoch 8/50
293/293 [==============================] - 246s 839ms/step - loss: 0.6932 - accuracy: 0.5021
Epoch 9/50
293/293 [==============================] - 249s 851ms/step - loss: 0.6932 - accuracy: 0.5012
Epoch 10/50
293/293 [==============================] - 241s 823m

ResourceExhaustedError:  OOM when allocating tensor with shape[64,64,224,224] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[node block1_conv2/convolution (defined at C:\Users\Arun\Anaconda3\lib\site-packages\tensorflow_core\python\framework\ops.py:1751) ]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.
 [Op:__inference_keras_scratch_graph_1770]

Function call stack:
keras_scratch_graph


In [ ]:
acc = model.evaluate_generator(validation, steps=len(validation))

In [1]:
from keras import backend as K
K.tensorflow_backend._get_available_gpus()

Using TensorFlow backend.


InternalError: cudaGetDevice() failed. Status: cudaGetErrorString symbol not found.

In [2]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 16270673869451703719
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 3061527347
locality {
  bus_id: 1
  links {
  }
}
incarnation: 13030324186648714553
physical_device_desc: "device: 0, name: GeForce GTX 1650, pci bus id: 0000:01:00.0, compute capability: 7.5"
]


In [2]:
import tensorflow as tf
tf.test.is_gpu_available( cuda_only=False, min_cuda_compute_capability=None )

False